In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 读取数据

In [ ]:
userLog = pd.read_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/user_log_format1.csv')
userInfo = pd.read_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/user_info_format1.csv')
train = pd.read_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/train_format1.csv')
test = pd.read_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/test_format1.csv')
submission = pd.read_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/sample_submission.csv')

In [ ]:
print('userLog dataset has {} rows and {} columns.'.format(userLog.shape[0],userLog.shape[1]))
print('userInfo dataset has {} rows and {} columns.'.format(userInfo.shape[0],userInfo.shape[1]))
print('train dataset has {} rows and {} columns.'.format(train.shape[0],train.shape[1]))
print('test dataset has {} rows and {} columns.'.format(test.shape[0],test.shape[1]))
print('submission dataset has {} rows and {} columns.'.format(submission.shape[0],submission.shape[1]))

userLog dataset has 54925330 rows and 7 columns.
userInfo dataset has 424170 rows and 3 columns.
train dataset has 260864 rows and 3 columns.
test dataset has 261477 rows and 3 columns.
submission dataset has 261477 rows and 3 columns.


In [ ]:
userLog.columns = ['user_id', 'item_id', 'cat_id', 'merchant_id', 'brand_id', 'time_stamp',
       'action_type']

In [ ]:
userLog = userLog.join(userInfo.set_index('user_id'), how='left', on='user_id')
userLog = userLog.join(train.set_index(['user_id','merchant_id']), how='left', on=['user_id','merchant_id'])
train = train.join(userInfo.set_index('user_id'), how='left', on='user_id')
test = test.join(userInfo.set_index('user_id'), how='left', on='user_id')
del userInfo

# EDA

## 基本描述

In [ ]:
print('User log contains {} rows and {} cols.'.format(userLog.shape[0],userLog.shape[1]))
print('There are {} unique items.'.format(len(userLog['item_id'].unique())))
print('There are {} unique categories.'.format(len(userLog['cat_id'].unique())))
print('There are {} unique sellers.'.format(len(userLog['merchant_id'].unique())))
print('There are {} unique brands.'.format(len(userLog['brand_id'].unique())))
print('There are {} unique time_tamp.'.format(len(userLog['time_stamp'].unique())))

User log contains 54925330 rows and 10 cols.
There are 1090390 unique items.
There are 1658 unique categories.
There are 4995 unique sellers.
There are 8444 unique brands.
There are 186 unique time_tamp.


In [ ]:
userLog['action_type'].value_counts()

0    48550713
2     3292144
3     3005723
1       76750
Name: action_type, dtype: int64

In [ ]:
userLog['age_range'].value_counts().sort_index()

0.0     9931162
1.0        1721
2.0     5385020
3.0    14848637
4.0    11802052
5.0     6200000
6.0     5413716
7.0     1052265
8.0      162534
Name: age_range, dtype: int64

In [ ]:
userLog['gender'].value_counts().sort_index()

0.0    40313178
1.0    12135530
2.0     2055127
Name: gender, dtype: int64

In [ ]:
train['label'].value_counts()

0    244912
1     15952
Name: label, dtype: int64

最少的用户购买了2件商品，最多的用户购买了14468件商品，平均购买了129件商品，中位数为74件商品，说明存在右偏。

In [ ]:
userLog.groupby('user_id')['item_id'].count().describe()

count    424170.000000
mean        129.488955
std         187.738160
min           2.000000
25%          37.000000
50%          74.000000
75%         150.000000
max       14468.000000
Name: item_id, dtype: float64

最多的Item销售了34万件，最少的Item销售了1件，平均销售了50件，中位数为7件，这说明存在右偏。

In [ ]:
userLog.groupby('item_id')['user_id'].count().describe()

count    1.090390e+06
mean     5.037219e+01
std      4.551030e+02
min      1.000000e+00
25%      2.000000e+00
50%      7.000000e+00
75%      2.700000e+01
max      3.459050e+05
Name: user_id, dtype: float64

## 特征工程

In [ ]:
userLog['month'] = userLog['time_stamp']//100
userLog['day'] = userLog['time_stamp']%100

In [ ]:
# Temp Helper Variables
userLog['action_type_0'] = userLog['action_type']==0
userLog['action_type_1'] = userLog['action_type']==1
userLog['action_type_2'] = userLog['action_type']==2
userLog['action_type_3'] = userLog['action_type']==3

In [ ]:
eval_list = {
  # User Feature
  '''pd.Series(name='user_interact_count',data=userLog.groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_click_count',data=userLog.groupby('user_id')['action_type_0'].aggregate('sum'))''':'user_id',
  '''pd.Series(name='user_cart_count',data=userLog.groupby('user_id')['action_type_1'].aggregate('sum'))''':'user_id',
  '''pd.Series(name='user_purchase_count',data=userLog.groupby('user_id')['action_type_2'].aggregate('sum'))''':'user_id',
  '''pd.Series(name='user_favourite_count',data=userLog.groupby('user_id')['action_type_3'].aggregate('sum'))''':'user_id',
  '''pd.Series(name='user_item_count',data=userLog.groupby('user_id')['item_id'].aggregate('nunique'))''':'user_id',
  '''pd.Series(name='user_cat_count',data=userLog.groupby('user_id')['cat_id'].aggregate('nunique'))''':'user_id',
  '''pd.Series(name='user_merchant_count',data=userLog.groupby('user_id')['merchant_id'].aggregate('nunique'))''':'user_id',
  '''pd.Series(name='user_brand_count',data=userLog.groupby('user_id')['brand_id'].aggregate('nunique'))''':'user_id',
  '''pd.Series(name='user_monthly_interact_count_mean',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').mean())''':'user_id',
  '''pd.Series(name='user_monthly_interact_count_std',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').std())''':'user_id',
  '''pd.Series(name='user_monthly_interact_count_max',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').max())''':'user_id',
  '''pd.Series(name='user_monthly_interact_count_min',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').min())''':'user_id',
  '''pd.Series(name='user_monthly_purchase_count_mean',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').mean())''':'user_id',
  '''pd.Series(name='user_monthly_purchase_count_std',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').std())''':'user_id',
  '''pd.Series(name='user_monthly_purchase_count_max',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').max())''':'user_id',
  '''pd.Series(name='user_monthly_purchase_count_min',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').min())''':'user_id',
  '''pd.Series(name='user_daily_interact_count_mean',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').mean())''':'user_id',
  '''pd.Series(name='user_daily_interact_count_std',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').std())''':'user_id',
  '''pd.Series(name='user_daily_interact_count_max',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').max())''':'user_id',
  '''pd.Series(name='user_daily_interact_count_min',data=userLog.groupby(['user_id','month'])['user_id'].aggregate('count').groupby('user_id').min())''':'user_id',
  '''pd.Series(name='user_daily_purchase_count_mean',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').mean())''':'user_id',
  '''pd.Series(name='user_daily_purchase_count_std',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').std())''':'user_id',
  '''pd.Series(name='user_daily_purchase_count_max',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').max())''':'user_id',
  '''pd.Series(name='user_daily_purchase_count_min',data=userLog.groupby(['user_id','month'])['action_type_3'].aggregate('sum').groupby('user_id').min())''':'user_id',
  '''pd.Series(name='user_month5_interact_count',data=userLog[userLog['month']==5].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_month6_interact_count',data=userLog[userLog['month']==6].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_month7_interact_count',data=userLog[userLog['month']==7].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_month8_interact_count',data=userLog[userLog['month']==8].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_month9_interact_count',data=userLog[userLog['month']==9].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_month10_interact_count',data=userLog[userLog['month']==10].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  '''pd.Series(name='user_month11_interact_count',data=userLog[userLog['month']==11].groupby('user_id')['user_id'].aggregate('count'))''':'user_id',
  
  # Merchant Feature
  '''pd.Series(name='merchant_interact_count',data=userLog.groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_click_count',data=userLog.groupby('merchant_id')['action_type_0'].aggregate('sum'))''':'merchant_id',
  '''pd.Series(name='merchant_cart_count',data=userLog.groupby('merchant_id')['action_type_1'].aggregate('sum'))''':'merchant_id',
  '''pd.Series(name='merchant_purchase_count',data=userLog.groupby('merchant_id')['action_type_2'].aggregate('sum'))''':'merchant_id',
  '''pd.Series(name='merchant_favourite_count',data=userLog.groupby('merchant_id')['action_type_3'].aggregate('sum'))''':'merchant_id',
  '''pd.Series(name='merchant_item_count',data=userLog.groupby('merchant_id')['item_id'].aggregate('nunique'))''':'merchant_id',
  '''pd.Series(name='merchant_cat_count',data=userLog.groupby('merchant_id')['cat_id'].aggregate('nunique'))''':'merchant_id',
  '''pd.Series(name='merchant_merchant_count',data=userLog.groupby('merchant_id')['merchant_id'].aggregate('nunique'))''':'merchant_id',
  '''pd.Series(name='merchant_brand_count',data=userLog.groupby('merchant_id')['brand_id'].aggregate('nunique'))''':'merchant_id',
  '''pd.Series(name='merchant_monthly_interact_count_mean',data=userLog.groupby(['merchant_id','month'])['merchant_id'].aggregate('count').groupby('merchant_id').mean())''':'merchant_id',
  '''pd.Series(name='merchant_monthly_interact_count_std',data=userLog.groupby(['merchant_id','month'])['merchant_id'].aggregate('count').groupby('merchant_id').std())''':'merchant_id',
  '''pd.Series(name='merchant_monthly_interact_count_max',data=userLog.groupby(['merchant_id','month'])['merchant_id'].aggregate('count').groupby('merchant_id').max())''':'merchant_id',
  '''pd.Series(name='merchant_monthly_interact_count_min',data=userLog.groupby(['merchant_id','month'])['merchant_id'].aggregate('count').groupby('merchant_id').min())''':'merchant_id',
  '''pd.Series(name='merchant_age_interact_count_mean',data=userLog.groupby(['merchant_id','age_range'])['merchant_id'].aggregate('count').groupby('merchant_id').mean())''':'merchant_id',
  '''pd.Series(name='merchant_age_interact_count_std',data=userLog.groupby(['merchant_id','age_range'])['merchant_id'].aggregate('count').groupby('merchant_id').std())''':'merchant_id',
  '''pd.Series(name='merchant_age_interact_count_max',data=userLog.groupby(['merchant_id','age_range'])['merchant_id'].aggregate('count').groupby('merchant_id').max())''':'merchant_id',
  '''pd.Series(name='merchant_age_interact_count_min',data=userLog.groupby(['merchant_id','age_range'])['merchant_id'].aggregate('count').groupby('merchant_id').min())''':'merchant_id',
  '''pd.Series(name='merchant_gender_interact_count_mean',data=userLog.groupby(['merchant_id','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').mean())''':'merchant_id',
  '''pd.Series(name='merchant_gender_interact_count_std',data=userLog.groupby(['merchant_id','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').std())''':'merchant_id',
  '''pd.Series(name='merchant_gender_interact_count_max',data=userLog.groupby(['merchant_id','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').max())''':'merchant_id',
  '''pd.Series(name='merchant_gender_interact_count_min',data=userLog.groupby(['merchant_id','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').min())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_interact_count_mean',data=userLog.groupby(['merchant_id','age_range','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').mean())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_interact_count_std',data=userLog.groupby(['merchant_id','age_range','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').std())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_interact_count_max',data=userLog.groupby(['merchant_id','age_range','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').max())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_interact_count_min',data=userLog.groupby(['merchant_id','age_range','gender'])['merchant_id'].aggregate('count').groupby('merchant_id').min())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_purchase_count_mean',data=userLog.groupby(['merchant_id','age_range','gender'])['action_type_2'].aggregate('sum').groupby('merchant_id').mean())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_purchase_count_std',data=userLog.groupby(['merchant_id','age_range','gender'])['action_type_2'].aggregate('sum').groupby('merchant_id').std())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_purchase_count_max',data=userLog.groupby(['merchant_id','age_range','gender'])['action_type_2'].aggregate('sum').groupby('merchant_id').max())''':'merchant_id',
  '''pd.Series(name='merchant_ageXgender_purchase_count_min',data=userLog.groupby(['merchant_id','age_range','gender'])['action_type_2'].aggregate('sum').groupby('merchant_id').min())''':'merchant_id',
  '''pd.Series(name='merchant_month5_interact_count',data=userLog[userLog['month']==5].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_month6_interact_count',data=userLog[userLog['month']==6].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_month7_interact_count',data=userLog[userLog['month']==7].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_month8_interact_count',data=userLog[userLog['month']==8].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_month9_interact_count',data=userLog[userLog['month']==9].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_month10_interact_count',data=userLog[userLog['month']==10].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',
  '''pd.Series(name='merchant_month11_interact_count',data=userLog[userLog['month']==11].groupby('merchant_id')['merchant_id'].aggregate('count'))''':'merchant_id',

  # User Merchant Feature
  '''pd.Series(name='user_merchant_interact_count',data=userLog.groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_click_count',data=userLog.groupby(['user_id','merchant_id'])['action_type_0'].aggregate('sum'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_cart_count',data=userLog.groupby(['user_id','merchant_id'])['action_type_1'].aggregate('sum'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_favourite_count',data=userLog.groupby(['user_id','merchant_id'])['action_type_3'].aggregate('sum'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_interact_count_mean',data=userLog.groupby(['user_id','merchant_id','month'])['merchant_id'].aggregate('count').groupby(['user_id','merchant_id']).mean())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_interact_count_std',data=userLog.groupby(['user_id','merchant_id','month'])['merchant_id'].aggregate('count').groupby(['user_id','merchant_id']).std())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_interact_count_max',data=userLog.groupby(['user_id','merchant_id','month'])['merchant_id'].aggregate('count').groupby(['user_id','merchant_id']).max())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_interact_count_min',data=userLog.groupby(['user_id','merchant_id','month'])['merchant_id'].aggregate('count').groupby(['user_id','merchant_id']).min())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_click_count_mean',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_0'].aggregate('sum').groupby(['user_id','merchant_id']).mean())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_click_count_std',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_0'].aggregate('sum').groupby(['user_id','merchant_id']).std())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_click_count_max',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_0'].aggregate('sum').groupby(['user_id','merchant_id']).max())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_click_count_min',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_0'].aggregate('sum').groupby(['user_id','merchant_id']).min())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_cart_count_mean',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_1'].aggregate('sum').groupby(['user_id','merchant_id']).mean())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_cart_count_std',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_1'].aggregate('sum').groupby(['user_id','merchant_id']).std())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_cart_count_max',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_1'].aggregate('sum').groupby(['user_id','merchant_id']).max())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_cart_count_min',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_1'].aggregate('sum').groupby(['user_id','merchant_id']).min())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_purchase_count_mean',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_2'].aggregate('sum').groupby(['user_id','merchant_id']).mean())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_purchase_count_std',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_2'].aggregate('sum').groupby(['user_id','merchant_id']).std())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_purchase_count_max',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_2'].aggregate('sum').groupby(['user_id','merchant_id']).max())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_purchase_count_min',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_2'].aggregate('sum').groupby(['user_id','merchant_id']).min())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_favourite_count_mean',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_3'].aggregate('sum').groupby(['user_id','merchant_id']).mean())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_favourite_count_std',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_3'].aggregate('sum').groupby(['user_id','merchant_id']).std())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_favourite_count_max',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_3'].aggregate('sum').groupby(['user_id','merchant_id']).max())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_monthly_favourite_count_min',data=userLog.groupby(['user_id','merchant_id','month'])['action_type_3'].aggregate('sum').groupby(['user_id','merchant_id']).min())''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month_count',data=userLog.groupby(['user_id','merchant_id'])['month'].aggregate('nunique'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_day_count',data=userLog.groupby(['user_id','merchant_id'])['time_stamp'].aggregate('nunique'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_item_count',data=userLog.groupby(['user_id','merchant_id'])['item_id'].aggregate('nunique'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_cat_count',data=userLog.groupby(['user_id','merchant_id'])['cat_id'].aggregate('nunique'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_merchant_count',data=userLog.groupby(['user_id','merchant_id'])['merchant_id'].aggregate('nunique'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_brand_count',data=userLog.groupby(['user_id','merchant_id'])['brand_id'].aggregate('nunique'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month5_interact_count',data=userLog[userLog['month']==5].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month6_interact_count',data=userLog[userLog['month']==6].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month7_interact_count',data=userLog[userLog['month']==7].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month8_interact_count',data=userLog[userLog['month']==8].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month9_interact_count',data=userLog[userLog['month']==9].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month10_interact_count',data=userLog[userLog['month']==10].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],
  '''pd.Series(name='user_merchant_month11_interact_count',data=userLog[userLog['month']==11].groupby(['user_id','merchant_id'])['merchant_id'].aggregate('count'))''':['user_id','merchant_id'],

  # Rate Feature
  '''pd.Series(name='user_interact_rate',data=userLog.groupby('user_id')['user_id'].aggregate('count')/userLog['user_id'].count())''' :'user_id',
  '''pd.Series(name='user_purchase_rate',data=userLog.groupby('user_id')['action_type_2'].aggregate('sum')/userLog['action_type_2'].sum())''' :'user_id',
  '''pd.Series(name='merchant_interact_rate',data=userLog.groupby('merchant_id')['merchant_id'].aggregate('count')/userLog['merchant_id'].count())''' :'user_id',
  '''pd.Series(name='merchant_purchase_rate',data=userLog.groupby('merchant_id')['action_type_2'].aggregate('sum')/userLog['action_type_2'].sum())''' :'user_id',
  '''pd.Series(name='merchant_user_interact_rate',data=userLog.groupby('merchant_id')['user_id'].aggregate('nunique')/userLog['user_id'].nunique())''' :'merchant_id',
  '''pd.Series(name='merchant_user_click_rate',data=userLog.groupby('merchant_id')['action_type_0'].aggregate('sum')/userLog['action_type_0'].sum())''' :'merchant_id',
  '''pd.Series(name='merchant_user_cart_rate',data=userLog.groupby('merchant_id')['action_type_1'].aggregate('sum')/userLog['action_type_1'].sum())''' :'merchant_id',
  '''pd.Series(name='merchant_user_purchase_rate',data=userLog.groupby('merchant_id')['action_type_2'].aggregate('sum')/userLog['action_type_2'].sum())''' :'merchant_id',
  '''pd.Series(name='merchant_user_favourite_rate',data=userLog.groupby('merchant_id')['action_type_3'].aggregate('sum')/userLog['action_type_3'].sum())''' :'merchant_id',
  '''pd.Series(name='user_month5_interact_rate',data=userLog[userLog['month']==5].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==5]['user_id'].count())''':'user_id',
  '''pd.Series(name='user_month6_interact_rate',data=userLog[userLog['month']==6].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==6]['user_id'].count())''':'user_id',
  '''pd.Series(name='user_month7_interact_rate',data=userLog[userLog['month']==7].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==7]['user_id'].count())''':'user_id',
  '''pd.Series(name='user_month8_interact_rate',data=userLog[userLog['month']==8].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==8]['user_id'].count())''':'user_id',
  '''pd.Series(name='user_month9_interact_rate',data=userLog[userLog['month']==9].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==9]['user_id'].count())''':'user_id',
  '''pd.Series(name='user_month10_interact_rate',data=userLog[userLog['month']==10].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==10]['user_id'].count())''':'user_id',
  '''pd.Series(name='user_month11_interact_rate',data=userLog[userLog['month']==11].groupby(['user_id'])['user_id'].aggregate('count')/userLog[userLog['month']==11]['user_id'].count())''':'user_id',
  '''pd.Series(name='merchant_month5_interact_rate',data=userLog[userLog['month']==5].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==5]['merchant_id'].count())''':'merchant_id',
  '''pd.Series(name='merchant_month6_interact_rate',data=userLog[userLog['month']==6].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==6]['merchant_id'].count())''':'merchant_id',
  '''pd.Series(name='merchant_month7_interact_rate',data=userLog[userLog['month']==7].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==7]['merchant_id'].count())''':'merchant_id',
  '''pd.Series(name='merchant_month8_interact_rate',data=userLog[userLog['month']==8].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==8]['merchant_id'].count())''':'merchant_id',
  '''pd.Series(name='merchant_month9_interact_rate',data=userLog[userLog['month']==9].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==9]['merchant_id'].count())''':'merchant_id',
  '''pd.Series(name='merchant_month10_interact_rate',data=userLog[userLog['month']==10].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==10]['merchant_id'].count())''':'merchant_id',
  '''pd.Series(name='merchant_month11_interact_rate',data=userLog[userLog['month']==11].groupby(['merchant_id'])['merchant_id'].aggregate('count')/userLog[userLog['month']==11]['merchant_id'].count())''':'merchant_id',
  ## Repurchase Rate
  '''pd.Series(name='merchant_repurchase_user_rate',data=userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')[userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')>1].groupby('merchant_id').aggregate('count')/userLog[userLog['action_type']==2].groupby('merchant_id')['user_id'].aggregate('nunique'))''': 'merchant_id',
  '''pd.Series(name='merchant_repurchase_count_rate',data=userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')[userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')>1].groupby('merchant_id').aggregate('sum')/userLog[userLog['action_type']==2].groupby('merchant_id')['user_id'].aggregate('count'))''' :'merchant_id',
  '''pd.Series(name='user_repurchase_merchant_rate',data=userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')[userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')>1].groupby('user_id').aggregate('sum')/userLog[userLog['action_type']==2].groupby('user_id')['merchant_id'].aggregate('nunique'))''' :'user_id',
  '''pd.Series(name='user_repurchase_count_rate',data=userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')[userLog.groupby(['user_id','merchant_id'])['action_type_2'].aggregate('sum')>1].groupby('user_id').aggregate('count')/userLog[userLog['action_type']==2].groupby('user_id')['merchant_id'].aggregate('count'))''' :'user_id'
  
}
for e, keys in eval_list.items():
  temp = eval(e)
  train = train.join(temp, on=keys)
  test = test.join(temp, on=keys)

## 填补缺失值为零

In [ ]:
test = test.drop(columns='prob')
train = train.fillna(0)
test = test.fillna(0)

# 写入文件

In [ ]:
print('train dataset has {} rows and {} columns.'.format(train.shape[0],train.shape[1]))
print('test dataset has {} rows and {} columns.'.format(test.shape[0],test.shape[1]))
train.to_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/train.csv')
test.to_csv('/content/drive/My Drive/Case/Repeat Buyers Prediction/data/test.csv')

train dataset has 260864 rows and 135 columns.
test dataset has 261477 rows and 134 columns.
